In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### **Section 1: Setup**

In [2]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


In [3]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("ADK component imported!!!")

ADK component imported!!!


In [4]:
retry_config=types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429,500,503,504]
)

## **creation multi agent sys**


## **Parallel Workflows - Independent Researchers**

- When you have independent tasks, you can run them all at the same time using a ParallelAgent.

- This agent executes all of its sub-agents concurrently, dramatically speeding up the workflow. Once all parallel tasks are complete, you can then pass their combined results to a final 'aggregator' step.

- *Use Parallel when: Tasks are independent, speed matters, and you can execute concurrently.*

- ![](https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/parallel-agent.png)

#### Parallel Multi-Topic Research¶
Let's build a system with four agents:

- Tech Researcher - Researches AI/ML news and trends
- Health Researcher - Researches recent medical news and trends
- Finance Researcher - Researches finance and fintech news and trends
- Aggregator Agent - Combines all research findings into a single summary

In [8]:
# Tech Researcher 
tech_researcher=Agent(
    name="TechResearch",
    model=Gemini(
        model='gemini-2.5-flash-lite',
        retry_option=retry_config
    ),
    instruction="""Research the latest AI/ML trends. Include 3 key developments,
the main companies involved, and the potential impact. Keep the report very concise (100 words).""",
    tools=[google_search],
    output_key="tech_research",  # The result of this agent will be stored in the session state with this key.
)

# Health Researcher
health_researcher=Agent(
    name="HealthResearch",
    model=Gemini(
        model='gemini-2.5-flash-lite',
        retry_option=retry_config
    ),
    instruction="""Research recent medical breakthroughs. Include 3 significant advances,
their practical applications, and estimated timelines. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="health_research", 
)

# Finance Researcher 
finance_researcher=Agent(
    name="FinanceResearch",
    model=Gemini(
        model='gemini-2.5-flash-lite',
        retry_option=retry_config
    ),
    instruction="""Research current fintech trends. Include 3 key trends,
their market implications, and the future outlook. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="finance_research", 
)

print("worker agents created...")

worker agents created...


In [11]:
# Aggregator Agent
# in {} bracket we write placeholders to inject the outputs from the parallel agents, which are now in the session state.eg:{agent_name_output_key}
aggregator_agent=Agent(
    name="AggregatorAgent",
    model=Gemini(
        model='gemini-2.5-flash-lite',
        retry_option=retry_config
    ),
    instruction="""Combine these three research findings into a single executive summary:

    **Technology Trends:**
    {tech_research}
    
    **Health Breakthroughs:**
    {health_research}
    
    **Finance Innovations:**
    {finance_research}
    
    Your summary should highlight common themes, surprising connections, and the most important key takeaways from all three reports. The final summary should be around 200 words.""",
    output_key="executive_summary", 
)
print("aggregator_agent created.")

aggregator_agent created.


In [12]:
# The ParallelAgent runs all its sub-agents simultaneously.
parallel_research_team= ParallelAgent(
    name='ParallelResearchTeam',
    sub_agents=[tech_researcher, health_researcher, finance_researcher]
)

# This SequentialAgent defines the high-level workflow: run the parallel team first, then run the aggregator.

root_agent=SequentialAgent(
    name='ResearchSystem',
    sub_agents=[parallel_research_team,aggregator_agent]
)

print("Parallel and Sequential Agents created...")

Parallel and Sequential Agents created...


In [13]:
runner=InMemoryRunner(agent=root_agent)
response=await runner.run_debug(
    "Run the daily executive briefing on Tech, Health, and Finance"
)


 ### Created new session: debug_session_id

User > Run the daily executive briefing on Tech, Health, and Finance
TechResearch > Here's a concise briefing on key AI/ML trends:

**Key AI/ML Developments for 2025:**

1.  **Generative AI Expansion:** Moving beyond content creation, Generative AI is now being applied to drug discovery, materials science, and complex problem-solving, with companies like **OpenAI**, **Google**, and **Microsoft** leading this charge.
2.  **Agentic AI and Automation:** AI agents capable of completing entire tasks with minimal human input are emerging, automating workflows in areas like scheduling and data analysis. **Google**, **Salesforce**, and **OpenAI** are prominent in this space.
3.  **Domain-Specific and Smaller AI Models:** Specialized AI models are increasingly outperforming general ones, offering greater efficiency and cost-effectiveness. This trend is driven by a need for more focused AI solutions across industries.

**Major Companies Involved:**
**

**This ts the final  output:**
- The rapid advancement and integration of Artificial Intelligence (AI) is the dominant theme across technology, health, and finance. Generative and agentic AI are moving beyond content creation to drive complex problem-solving in drug discovery, scientific research, and workflow automation. This surge in AI necessitates increased demand for computing power and cybersecurity measures, especially with the rise of autonomous systems and the Internet of Medical Things (IoMT).

- In healthcare, AI is revolutionizing diagnostics, exceeding human accuracy in disease detection, and will accelerate gene editing therapies and the development of non-opioid pain relief. Similarly, finance is leveraging AI and ML for fraud reduction, with new payment technologies like stablecoins and real-time bank transfers gaining traction, alongside the expansion of embedded finance and decentralized finance (DeFi).

- Key takeaways include the pervasive impact of AI across all sectors, driving efficiency and innovation. Emerging technologies like quantum computing and Wi-Fi 7 are also poised to transform industries. The interconnectedness of these advancements highlights a future where AI acts as a creative collaborator, accelerating scientific breakthroughs and fundamentally reshaping how work and services are delivered.